In [105]:
import pandas as pd
import numpy as np
import torch


from torch import nn, optim
import torch.nn.functional as F

In [106]:
x_dataset = pd.read_pickle("./x_dataset.pkl")
y_dataset = pd.read_pickle("./y_dataset.pkl")


In [107]:
del x_dataset["id"]
del y_dataset["user_id"]

In [108]:
y_dataset = y_dataset.astype('float64')

In [109]:
y_dataset.dtypes

ae264e3637204a6fb9bb56bc8210ddfd    float64
4d5c57ea9a6940dd891ad53e9dbe8da0    float64
3f207df678b143eea3cee63160fa8bed    float64
9b98b8c7a33c4b65b9aebfe6a799e6d9    float64
0b1e1539f2cc45b7b9fa7c272da2e1d7    float64
2298d6c36e964ae4a3e7e9706d1fb8c2    float64
fafdcd668e3743c1bb461111dcafc2a4    float64
5a8bc65990b245e5a138643cd4eb9837    float64
f19421c1d4aa40978ebb69ca19b0e20d    float64
2906b810c7d4411798c6938adc9daaa5    float64
dtype: object

In [110]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x_dataset, y_dataset, test_size = 0.2, random_state = 0)

In [113]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [114]:
class Classifier(nn.Module):
    def __init__(self, input_features, hidden_dim_1, hidden_dim_2, output_dim):
        """
        Initialize the model by setting up linear layers.
        Use the input parameters to help define the layers of your model.
        :param input_features: the number of input features in your training/test data
        :param hidden_dim: helps define the number of nodes in the hidden layer(s)
        :param output_dim: the number of outputs you want to produce
        """
        super().__init__()
        
        #Fully conected
        self.fc1 = nn.Linear(input_features, hidden_dim_1)
        self.fc2 = nn.Linear(hidden_dim_1, hidden_dim_2)
        self.fc3 = nn.Linear(hidden_dim_2, output_dim)
       
         # Droput 
        self.drop = nn.Dropout(0.3)
        
        # Softmax Layer
        self.soft = nn.Softmax()
        
        
    def forward(self, x):
        """
        Perform a forward pass of our model on input features, x.
        :param x: A batch of input features of size (batch_size, input_features)
        :return: A single, sigmoid-activated value as output
        """
        
        # feedforward behavior
        
        out = F.relu(self.fc1(x))
        out = self.drop(out)
        out = F.relu(self.fc2(out))
        out = self.drop(out)
        out = self.fc3(out)
        
        return self.soft(out)
        

In [115]:
model = Classifier(xTrain.shape[1], 16, 32, yTrain.shape[1]).to(device)
criterion = nn.MultiLabelSoftMarginLoss()
criterion= nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [99]:
%%time

epochs = 50
model = model.double()
for e in range(epochs):
    running_loss = 0
    for x, y in zip(xTrain.values, yTrain.values):
        x = torch.from_numpy(x).to(device)
        y = torch.from_numpy(y).to(device)
        log_ps = model(x)

        loss = criterion(log_ps, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(xTrain)}")

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Training loss: 0.7269590519171764
Training loss: 0.7266393195876225
Training loss: 0.7265921910266977
Training loss: 0.7265339942221624
Training loss: 0.7265582475040683
Training loss: 0.7265190638626722
Training loss: 0.7265372266689203
Training loss: 0.7265143095030331
Training loss: 0.7265213423907252
Training loss: 0.726529092702399
Training loss: 0.7265283122613779
Training loss: 0.7265332564785395
Training loss: 0.726546058688633
Training loss: 0.7265425674086434
Training loss: 0.7264847116571415
Training loss: 0.7264839995037864
Training loss: 0.7264834834031135
Training loss: 0.72651325330455
Training loss: 0.7265239917088993
Training loss: 0.7265337550720186
Training loss: 0.7265005472850116
Training loss: 0.7265036763469175
Training loss: 0.7264763486384143
Training loss: 0.7265022538385308
Training loss: 0.7264920685025649
Training loss: 0.7264948570252314
Training loss: 0.7265226235453008
Training loss: 0.7265237361428586
Training loss: 0.7265060515234072
Training loss: 0.7